<a href="https://colab.research.google.com/github/ovozzx/Machine-Learning/blob/master/%5BDacon%5DTitanic_Survivor_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 라이브러리(필요한 도구) 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score

import warnings
warnings.filterwarnings("ignore")

# 1. 데이터 로딩

PassengerID : 탑승객 고유 아이디

Survival : 탑승객 생존 유무 (0: 사망, 1: 생존)

Pclass : 등실의 등급

Name : 이름

Sex : 성별

Age : 나이

Sibsp : 함께 탐승한 형제자매, 아내, 남편의 수

Parch : 함께 탐승한 부모, 자식의 수

Ticket :티켓 번호

Fare : 티켓의 요금

Cabin : 객실번호

Embarked : 배에 탑승한 항구 이름 ( C = Cherbourn, Q = Queenstown, S = Southampton)



✓ 생존확률(0: 사망, 1: 생존)


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 2. 데이터 전처리

1) 표준화

2) 이상치 발견 및 제거 (z-score 활용)

3) 정규화  

In [ ]:
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [ ]:
train.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


###✓describe()

숫자형 (int, float) column들의 기술 통계량을 보여주는 함수

-기술통계량이란?

해당 column을 대표할 수 있는 통계값들을 의미

-기술통계량 종류

count: 해당 column에서 비어 있지 않은 값의 개수

mean: 평균

std: 표준편차

min: 최솟값 (이상치 포함)

25% (Q1): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 1/4번째 지점에 있는 값

50% (Q2): 중앙값 (전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 2/4번째 지점에 있는 값)

75% (Q3): 전체 데이터를 순서대로 정렬했을 때, 아래에서 부터 3/4번째 지점에 있는 값

max: 최댓값 (이상치 포함)

-이상치: 울타리 밖에 있는 부분을 이상치라고 정의함

아래쪽 울타리: Q_1Q1​ - 1.5 * IQR1.5∗IQR

위쪽 울타리: Q_3Q3​ + 1.5 * IQR1.5∗IQR

IQRIQR = Q_3 - Q_1Q3​−Q1​


In [ ]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


###✓pd.DataFrame.fillna()

결측치를 채우고자 하는 column과 결측치를 대신하여 넣고자 하는 (값)을 명시해주어야 합니다.

범주형 변수일 경우, 최빈값으로 대체할 수 있습니다.


###✓pd.DataFrame.map({딕셔너리})
딕셔너리에 따라 key값을v value로 바꿈

##✓정규화

특성 값의 범위를 [0, 1]로 옮김

(X - MIN) / (MAX-MIN) 


##✓표준화(Z 점수 정규화)

어떤 특성의 값들이 정규분포, 즉 종모양의 분포를 따른다고 가정하고 값들을 0의 평균, 1의 표준편차를 갖도록 변환, 이상치 잘 처리

(X - 평균) / 표준편차

In [ ]:
train['Age'] = train['Age'].fillna(29) #최빈값
train['Sex'] = train['Sex'].map({'male' : 0, 'female' : 1})

test['Age'] = test['Age'].fillna(29)
test['Sex'] = test['Sex'].map({'male' : 0, 'female' : 1})

In [ ]:
#Embarked 데이터를 숫자형으로 변환합니다.
"""
train['Embarked'] = train['Embarked'].fillna(0)
train['Embarked'] = train['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int)
test['Embarked'] = test['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int)
"""

"\ntrain['Embarked'] = train['Embarked'].fillna(0)\ntrain['Embarked'] = train['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int)\ntest['Embarked'] = test['Embarked'].map( {'C': 0, 'Q': 1, 'S':2} ).astype(int)\n"

In [ ]:
train = train.drop(['Name', 'Ticket'], axis = 1)
test = test.drop(['Name', 'Ticket'], axis = 1)

In [ ]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,1,0,3,0,22.0,1,0,7.2500,NaN,S
1,2,1,1,1,38.0,1,0,71.2833,C85,C
2,3,1,3,1,26.0,0,0,7.9250,NaN,S
3,4,1,1,1,35.0,1,0,53.1000,C123,S
4,5,0,3,0,35.0,0,0,8.0500,NaN,S


In [ ]:
train_x = train[['Pclass', 'SibSp','Age', 'Sex']]
train_y = train['Survived']


test_x = test[['Pclass', 'SibSp', 'Age', 'Sex']]

In [ ]:
##표준화
from sklearn.preprocessing import StandardScaler


# StandardScaler객체 생성
scaler = StandardScaler()
# StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
scaler.fit(train_x)
train_scaled = scaler.transform(train_x)
train_x = train_scaled

scaler.fit(test_x)
test_scaled = scaler.transform(test_x)
test_x = test_scaled

print(train_scaled)





[[ 0.82737724  0.43279337 -0.58165904 -0.73769513]
 [-1.56610693  0.43279337  0.64932701  1.35557354]
 [ 0.82737724 -0.4745452  -0.27391253  1.35557354]
 ...
 [ 0.82737724  0.43279337 -0.04310264  1.35557354]
 [-1.56610693 -0.4745452  -0.27391253 -0.73769513]
 [ 0.82737724 -0.4745452   0.18770724 -0.73769513]]


In [ ]:
##정규화

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler객체 생성
scaler = MinMaxScaler()
# MinMaxScaler 로 데이터 셋 변환. fit() 과 transform() 호출.  
scaler.fit(train_x)
train_scaled = scaler.transform(train_x)

train_x = train_scaled 



scaler.fit(test_x)
test_scaled = scaler.transform(test_x)

test_x = test_scaled 


In [ ]:
train_x.shape

(891, 4)

In [ ]:
train_y.shape

(891,)

# 3. 모델 정의 및 학습

- 의사결정나무 : 정확도 0.70435

- **로지스틱회귀** : 정확도 0.7527507303

- 랜덤포레스트 : 정화도 0.7024342746

- **LightGBM** : 정확도 0.7377312561

- **딥러닝** : 정확도 0.756037001

(1) 의사결정나무

In [ ]:
#0.70435

# 모델 만들기
DT_model = DecisionTreeClassifier()
# 학습하기
DT_model.fit(train_x, train_y)

DecisionTreeClassifier()

In [ ]:
DT_test_y_pred = DT_model.predict(test_x)
DT_test_y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

(2) 로지스틱회귀 : 0.7527507303

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’


In [ ]:

#0.73984
    

LR_model = LogisticRegression(random_state = 10, solver='liblinear')

LR_model.fit(train_x, train_y)

LogisticRegression(random_state=10, solver='liblinear')

In [ ]:
LR_model.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

- penalty : 정규화의 방법 지정 → l1, l2

- C : 정규화의 강도 지정(작을수록 강도 증가)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

params = { 'penalty' : ['l2', 'l1'],
           'C' : [0.01,0.1, 1, 3, 7, 8,10],
           'max_iter' : [3,4,5,10,20,30,50,100,200]
         }


grid_LR = GridSearchCV(LR_model, param_grid = params, scoring = 'accuracy', cv =3 )
grid_LR.fit(train_x, train_y)
print('최적 하이퍼 파라미터:{0}, 최적 평균 정확도:{1:.3f}'.format(grid_LR.best_params_, grid_LR.best_score_))

최적 하이퍼 파라미터:{'C': 7, 'max_iter': 5, 'penalty': 'l1'}, 최적 평균 정확도:0.802
CPU times: user 1.03 s, sys: 2.9 ms, total: 1.04 s
Wall time: 1.05 s


In [ ]:
#0.7527507303
LR_test_y_pred = grid_LR.predict(test_x)
LR_test_y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

(3) 랜덤포레스트

In [ ]:
#0.7024342746

# 모델 만들기
RF_model = RandomForestClassifier()
# 학습하기
RF_model.fit(train_x, train_y)

RandomForestClassifier()

In [ ]:
RF_test_y_pred = DT_model.predict(test_x)
RF_test_y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,

(4) LightGBM  : 0.7377312561

In [ ]:
#0.7240262902

# 모델 만들기
LG_model = LGBMClassifier()
# 학습하기
LG_model.fit(train_x, train_y)

LGBMClassifier()

In [ ]:
LG_model.get_params().keys()

dict_keys(['boosting_type', 'class_weight', 'colsample_bytree', 'importance_type', 'learning_rate', 'max_depth', 'min_child_samples', 'min_child_weight', 'min_split_gain', 'n_estimators', 'n_jobs', 'num_leaves', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'silent', 'subsample', 'subsample_for_bin', 'subsample_freq'])

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

# 하이퍼 파라미터 테스트의 수행 속도를 향상시키기 위해 n_estimators를 200으로 감소 → 실제 학습시킬 때 증가시킴
LG_model = LGBMClassifier(random_state = 10)

params = {'max_depth':[64, 128, 160, 200] , 
          'min_child_samples':[20,30, 60,100] ,
          'num_leaves':[8,16, 32, 64], 
          'subsample':[0,5, 0.8, 1],
          'n_estimators' : [60,100,200,300]
          }

# cv는 3으로 지정 
grid_LG = GridSearchCV(LG_model, param_grid=params, cv=3)
grid_LG.fit(train_x, train_y)


print('최적 하이퍼 파라미터:{0}, \n 최적 평균 정확도:{1:.3f}'.format(grid_LG.best_params_, grid_LG.best_score_))

최적 하이퍼 파라미터:{'max_depth': 64, 'min_child_samples': 20, 'n_estimators': 60, 'num_leaves': 8, 'subsample': 0.8}, 
 최적 평균 정확도:0.829
CPU times: user 1min 50s, sys: 7.57 s, total: 1min 57s
Wall time: 1min 2s


In [ ]:
LG_test_y_pred = grid_LG.predict(test_x)
LG_test_y_pred

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

(5) 딥러닝 : 0.756037001

In [ ]:
#1. 딥러닝
#2. 검색(캐글)
#3. 블로그 -> 포폴



In [ ]:
train_y.class

AttributeError: ignored

In [ ]:

from tensorflow.keras.utils import to_categorical

train_y = to_categorical(train_y)
train_y 


#원핫인코딩 0 1 1...

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf

DL = models.Sequential(name = 'Classification')
DL.add(layers.Dense(64, activation = 'relu', input_shape = (4,))) #특성 13개 input
DL.add(layers.Dense(32, activation = 'relu'))
DL.add(layers.Dense(16, activation = 'relu'))
DL.add(layers.Dense(2, activation = 'sigmoid'))

In [ ]:
DL.compile(loss = 'binary_crossentropy',
               optimizer = tf.keras.optimizers.Adam(lr=0.001),
               metrics = ['accuracy'])

In [ ]:
%%time

Hist_DL = DL.fit(train_x, train_y, epochs = 200, batch_size = 1)
                         

Epoch 1/200
891/891 [==============================] - 2s 2ms/step - loss: 0.4963 - accuracy: 0.7969
Epoch 2/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4507 - accuracy: 0.7890
Epoch 3/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4512 - accuracy: 0.7912
Epoch 4/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4450 - accuracy: 0.7969
Epoch 5/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4424 - accuracy: 0.7980
Epoch 6/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4389 - accuracy: 0.8070
Epoch 7/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4348 - accuracy: 0.8103
Epoch 8/200
891/891 [==============================] - 2s 2ms/step - loss: 0.4309 - accuracy: 0.7969
Epoch 9/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4331 - accuracy: 0.8013
Epoch 10/200
891/891 [==============================] - 1s 2ms/step - loss: 0.4263 - accura

In [ ]:
DL_test_y_pred = DL.predict(test_x)
DL_test_y_pred

array([[8.79438698e-01, 1.20562583e-01],
       [6.17937446e-01, 3.82045329e-01],
       [8.99952173e-01, 1.00050211e-01],
       [8.64279628e-01, 1.35720730e-01],
       [3.48015666e-01, 6.51982725e-01],
       [8.33052576e-01, 1.66945636e-01],
       [3.13845158e-01, 6.86154783e-01],
       [8.85449290e-01, 1.14552319e-01],
       [2.27690309e-01, 7.72313058e-01],
       [8.99757624e-01, 1.00244641e-01],
       [8.68472457e-01, 1.31528109e-01],
       [6.52880192e-01, 3.47102344e-01],
       [7.60623813e-03, 9.92394328e-01],
       [9.35685992e-01, 6.43174350e-02],
       [8.04573298e-03, 9.91954863e-01],
       [4.19274867e-02, 9.58073437e-01],
       [8.92349362e-01, 1.07652605e-01],
       [8.51021945e-01, 1.48977548e-01],
       [3.90603721e-01, 6.09392583e-01],
       [5.38855135e-01, 4.61128891e-01],
       [6.52880192e-01, 3.47102344e-01],
       [2.53405988e-01, 7.46602952e-01],
       [5.30186296e-03, 9.94698524e-01],
       [6.52880192e-01, 3.47102344e-01],
       [7.901698

In [ ]:
arr = []

for i in DL_test_y_pred :
  if i[0] > i[1] :
    arr.append(0)
  else :
    arr.append(1)    

DL_test_y_pred = arr

#성능 평가

# 4. 제출 파일 생성

In [ ]:
submission = pd.read_csv("submission.csv")
submission['Survived'] = DL_test_y_pred
submission.to_csv('DL_submission.csv', index = False)